In [1]:
def style(cell):
    """style(n)
    Funkcja sprawdzająca zgodność komórki `In[n]` notatnika
    ze standardem stylu PEP-8."""
    try:
        import pycodestyle as pep8
    except ImportError:
        import pep8
    code = (In[cell]+'\n').splitlines(True)
    return pep8.Checker('In[{0}]'.format(cell), code).check_all()

In [4]:
import string


def zaszyfruj(wiadomosc, klucz):
    '''
    Funkcja przygotowująca zaszyfrowaną wiadomość zgodnie
    z szyfrem Vigenera.
    *******************************************
    Argumenty:
    wiadomosc (str): napis o dowolnej dlugosci,
        który zostanie zaszyfrowany.
    klucz (str): klucz szyfrowania; w przypadku
        klucza krótszego niż wiadomość zostanie
        on rozszerzony do długości wiadomości.
    *******************************************
    Testy:

    >>> zaszyfruj('ABRAKADABRA', 'PIES')
    'PJVSZIHSQZE'

    >>> zaszyfruj('PROGRAMOWANIE','SWITALA')
    'HNWZRLMGSIGIP'
    '''
    litery = list(string.ascii_uppercase)
    wiadomosc = wiadomosc.upper()
    klucz = klucz.upper()
    w = len(wiadomosc)
    k = len(klucz)
    klucz_dl = (w//k)*klucz + klucz[:(w % k)]
    szyfr = ''
    for i in range(w):
        W = wiadomosc[i]
        s = litery.index(klucz_dl[i])
        szyfr = szyfr + chr((ord(W) + s-65) % 26 + 65)
    return szyfr


def deszyfruj(szyfr, klucz):
    '''
    Funkcja przygotowująca oryginalną wiadomość
    z szyfrogramu przygotowanego zgodnie z szyfrem
    Vigenera.
    *******************************************
    Argumenty:
    wiadomosc (str): napis o dowolnej dlugosci,
        który zostanie odszyfrowany.
    klucz (str): klucz szyfrowania; w przypadku
        klucza krótszego niż wiadomość zostanie
        on rozszerzony do długości wiadomości.
    *******************************************
    Testy:

    >>> deszyfruj('PJVSZIHSQZE', 'PIES')
    'ABRAKADABRA'

    >>> deszyfruj('HNWZRLMGSIGIP','SWITALA')
    'PROGRAMOWANIE'
    '''
    litery = list(string.ascii_uppercase)
    szyfr = szyfr.upper()
    klucz = klucz.upper()
    z = len(szyfr)
    k = len(klucz)
    klucz_dl = (z//k)*klucz + klucz[:(z % k)]
    wiadomosc = ''
    for i in range(z):
        c = szyfr[i]
        s = litery.index(klucz_dl[i])
        wiadomosc = wiadomosc + chr((ord(c) - s+65) % 26 + 65)
    return wiadomosc

In [8]:
szyfr = zaszyfruj('ABRAKADABRA', 'PIES')
wiadomosc = deszyfruj(szyfr, 'PIES')

%doctest_mode
import doctest
doctest.testmod()
%doctest_mode


print('Zaszyfrowana wiadomość: ', szyfr)
print('Odszyfrowana wiadomość: ', wiadomosc)

Exception reporting mode: Plain
Doctest mode is: ON
Exception reporting mode: Context
Doctest mode is: OFF
Zaszyfrowana wiadomość:  PJVSZIHSQZE
Odszyfrowana wiadomość:  ABRAKADABRA


In [12]:
def porównaj(freq1, freq2):
    delta = 0
    for litera, częstość in freq1.items():
        if litera not in freq2:
            delta += częstość
        else:
            delta += abs(częstość - freq2[litera])
    for litera, częstość in freq2.items():
        if litera not in freq1:
            delta += częstość
    return delta


freq = {
  'A': 0.099,  'B': 0.0147, 'C': 0.0436, 'D': 0.0325,
  'E': 0.0877, 'F': 0.003,  'G': 0.0142, 'H': 0.0108,
  'I': 0.0821, 'J': 0.0228, 'K': 0.0351, 'L': 0.0392,
  'M': 0.028,  'N': 0.0572, 'O': 0.086,  'P': 0.0313,
  'Q': 0.0014, 'R': 0.0469, 'S': 0.0498, 'T': 0.0398,
  'U': 0.025,  'V': 0.004,  'W': 0.0465, 'X': 0.0002,
  'Y': 0.0376, 'Z': 0.0653
}


def zloz_wiadomosc(odkodowane, l):
    '''
    Funkcja, która przeszukuje i porządkuje podany ciag
    znaków skacząc między znakami o długość klucza
    użytego do zaszyfrowania wiadomości.
    *******************************************
    Argumenty
    odkodowane (str): ciąg odkodowanych znaków z otrzymanej,
        zakodowanej wiadomości.
    l (int): dlugosc klucza użytego do zaszyfrowania wiadomosci
    '''
    tekst = ''
    for o in odkodowane:
        tekst += ''.join(o)
    uporzadkowany = []
    i, j, s = 0, 0, len(odkodowane[0])
    while i != l:
        uporzadkowany.append(tekst[j])
        j += s
        if j > len(tekst):
            j = j % len(tekst)
        elif j == len(tekst):
            break
        i += 1
    wiadomosc = ''.join(uporzadkowany)
    return wiadomosc


def deszyfruj_klucz(wiadomosc, dl_klucza):
    '''
    Funkcja, której zadaniem jest odszyfrowanie wiadomości,
    wiedząc, że użyty został szyf Vigenera i znajać długość
    użytego klucza. Zwraca odkowaną wiadomość oraz
    najprawdopodobniejszy przypadek użytego klucza.
    *******************************************
    Argumenty:
    wiadomosc (str): zakodowana wiadomość z użyciem szyfru
        Vigenera.
    dl_klucza (int): długość użytego klucza.
    '''
    czesci = []
    for i in range(dl_klucza):
        czesci.append(wiadomosc[i::dl_klucza])
    wynik_porownania = {}
    znaki = []
    caly_klucz = ''
    for czesc in czesci:
        for i in range(0, 26):
            odkodowana = deszyfruj(czesc.lower().replace('\n', ''), chr(i+65))
            odkodowana = odkodowana.upper()
            czestosc_liter = {v: odkodowana.count(v)/len(odkodowana)
                              for v in set(odkodowana)}
            wynik_porownania[i] = porównaj(freq, czestosc_liter)
        klucz = chr(min(wynik_porownania, key=wynik_porownania.get)+65)
        caly_klucz += klucz
        znaki.append(list(deszyfruj(czesc, klucz)))
    odkryte_haslo = zloz_wiadomosc(znaki, len(wiadomosc))
    return odkryte_haslo, caly_klucz

In [18]:
wiadomosc = '''FTTQSFBERJLEFDMGUCABSAFEDQVXFBTEYQNBXGBHBFWDTGVMPAVTNEZBVHMLIABRUOGYWXERXRQRDRCUVRLFAHEQEZBRXVBDYHLXMOSADGLQLIPBDHHDHATEBUQEQGSLTBQQHEDCHQOXVSCHHNVZLFRQVTFRTSMRDFRWKDLMFBWKWAHHDIZVBPFHJRWMROTHBEATVTWKYOXVSCHKABSSXRHYTTHWLEATRXVBRNFCEYNSGTDHXVEZXYHLDRN'''
odkodowana_wiadomosc, klucz = deszyfruj_klucz(wiadomosc, 7)
print('Odkodowana wiadomość brzmi następująco: ', odkodowana_wiadomosc)
print('Klucz szyfrowania: ', klucz)

Odkodowana wiadomość brzmi następująco:  BNOOSACIAPAAAALEJEA
Klucz szyfrowania:  EGFCAFB
